In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix , f1_score, accuracy_score,precision_score , recall_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
import joblib

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/urgensingtan/Amnilinternship/venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <E8D72161-CCD1-3423-9388-36D4CA0A7524> /Users/urgensingtan/Amnilinternship/venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/l

In [7]:
data = pd.read_csv('finalized_preprocessed_data.csv')

In [8]:
X = data.drop(columns='is_fraud')
y = data['is_fraud']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


In [9]:
#doing XG boost implementation
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [ ]:
parameters_grid = {
    'n_estimators': [300, 1500],
    'max_depth': [4,10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],

}
xgboost_model = XGBClassifier(**parameters_grid)
random_search = RandomizedSearchCV(estimator=xgboost_model, param_distributions=parameters_grid, n_iter=100, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
y_pred = random_search.predict(X_test)

In [ ]:
xgbmodel = joblib.dump(random_search, 'xgboost_model.pkl')

In [ ]:
y_pred = xgbmodel.predict(X_test)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
confusion_matrix(y_test, y_pred)


metric = {
    "f1_score": f1,
    "precision": precision,
    "recall": recall}